In [54]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name = "quick-starts-ws-131674")
exp = Experiment(workspace=ws, name = "udacity-project-1")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-131674
Azure region: southcentralus
Subscription id: a24a24d5-8d87-4c8a-99b6-91ed2d2df51f
Resource group: aml-quickstarts-131674


In [55]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
try:
    aml_compute = ComputeTarget(workspace = ws, name = "MainCompute")
    print("Found existing...")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size = 'Standard_D2_V2', max_nodes = 4)
    aml_compute = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)
    
aml_compute.wait_for_completion(show_output = True)

Found existing...

Running


In [56]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, quniform
import os

# Specify parameter sampler
ps = RandomParameterSampling ({'--C': uniform(0.1, 1), 
                              '--max_iter': quniform(100, 1500,100),})
### YOUR CODE HERE ###

# Specify a Policy
policy = BanditPolicy(evaluation_interval = 2, slack_factor = 0.1) ### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn("./training", compute_target= aml_compute, entry_script = "train.py") ### YOUR CODE HERE ###

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
                            estimator = est,
                            hyperparameter_sampling = ps, 
                            #policy = policy, 
                            primary_metric_name = 'Accuracy',
                            primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                            max_total_runs = 12,
                            max_concurrent_runs = 4,)

In [57]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
run_hyperdrive = exp.submit(config = hyperdrive_config)

In [58]:

RunDetails(run_hyperdrive).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [ ]:
import joblib
# Get your best run and save the model from that run.

best_run = run_hyperdrive.get_best_run_by_primary_metric()

print(best_run.get_details()['runDefinition']['arguments'])
### YOUR CODE HERE ###

In [ ]:
model = best_run.register_model(model_name = 'bankmarketing-logit', model_path = 'outputs/bankmarketing-logit-model.joblib')
model.download(target_dir = 'outputs', exist_ok = True)

## AutoML

In [61]:
exp_automl = Experiment(workspace=ws, name="udacity-project-1-automl")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: quick-starts-ws-131674
Azure region: southcentralus
Resource group: aml-quickstarts-131674


In [62]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
datastore = ws.get_default_datastore()
factory = TabularDatasetFactory()
data_path_train = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
###data_path_valid = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_validate.csv"
###data_path_test = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_test.csv"


ds_train = factory.from_delimited_files(data_path_train)
###ds_valid = factory.from_delimited_files(data_path_valid)
###ds_test = factory.from_delimited_files(data_path_test)

KeyboardInterrupt: 

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds_train)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=ds_train,
    label_column_name='y',
    n_cross_validations=2)

In [ ]:
# Submit your automl run

remote_run = exp_automl.submit(automl_config, show_output = False)

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###